In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
import cooler
import cooltools.lib.plotting
from cooltools import insulation

import os
import sys

# local imports 
import filters
import pore_c_utils as pcu

In [2]:
dirpath = "/nfs/turbo/umms-indikar/shared/tools/Pore-C-Snakemake/results_ES5to12/matrix/"
cell = "run07"

filepath = f"{dirpath}NlaIII_{cell}_GRCm39_unphased.matrix.mcool"
print(filepath)

/nfs/turbo/umms-indikar/shared/tools/Pore-C-Snakemake/results_ES5to12/matrix/NlaIII_run07_GRCm39_unphased.matrix.mcool


In [3]:
cooler.fileops.list_coolers(filepath)

['/resolutions/1000',
 '/resolutions/2000',
 '/resolutions/5000',
 '/resolutions/10000',
 '/resolutions/25000',
 '/resolutions/50000',
 '/resolutions/100000',
 '/resolutions/250000',
 '/resolutions/500000',
 '/resolutions/1000000',
 '/resolutions/2500000',
 '/resolutions/5000000',
 '/resolutions/10000000']

In [4]:
# KR balancing - note that this changes the file

resolution = 1000000

clr = cooler.Cooler(f'{filepath}::resolutions/{resolution}')
bias = cooler.balance_cooler(clr)

clr.bins()[:]['weight'] = bias[0]

INFO:cooler.balance:variance is 7031314884.124446
INFO:cooler.balance:variance is 3601785029.051504
INFO:cooler.balance:variance is 306235764.2233154
INFO:cooler.balance:variance is 3080839518.609856
INFO:cooler.balance:variance is 212646291.41995454
INFO:cooler.balance:variance is 2462017406.434736
INFO:cooler.balance:variance is 145303194.2959023
INFO:cooler.balance:variance is 1973286394.9391477
INFO:cooler.balance:variance is 100092882.52660324
INFO:cooler.balance:variance is 1577219642.7107773
INFO:cooler.balance:variance is 70641481.79298982
INFO:cooler.balance:variance is 1261410560.5417926
INFO:cooler.balance:variance is 51360119.49453666
INFO:cooler.balance:variance is 1013191763.587004
INFO:cooler.balance:variance is 38513581.17051718
INFO:cooler.balance:variance is 819111036.2902248
INFO:cooler.balance:variance is 29770881.283502556
INFO:cooler.balance:variance is 667172522.5683811
INFO:cooler.balance:variance is 23691473.213244632
INFO:cooler.balance:variance is 547644438.3

In [7]:
clr.bins()[:]

,chrom,start,end
0,NC_000067.7,0,1000000
1,NC_000067.7,1000000,2000000
2,NC_000067.7,2000000,3000000
3,NC_000067.7,3000000,4000000
4,NC_000067.7,4000000,5000000
...,...,...,...
2769,NT_166450.1,0,47073
2770,NT_166452.1,0,20208
2771,NT_187064.1,0,114452
2772,NW_023337853.1,0,31129


In [6]:
break

SyntaxError: 'break' outside loop (<ipython-input-6-6aaf1f276005>, line 1)